In [1]:
from IPython.core.magic import register_cell_magic


@register_cell_magic
def typecheck(line, cell):
    from IPython import get_ipython
    from mypy import api

    cell = "\n" + cell
    mypy_result = api.run(["-c", cell] + line.split())

    if mypy_result[0]:  # print mypy stdout
        print(mypy_result[0])

    if mypy_result[1]:  # print mypy stderr
        print(mypy_result[1])

    shell = get_ipython()
    shell.run_cell(cell)

In [2]:
import dataclasses

# Data Classesをつかってみよう
---
PyLadies Tokyo - 4周年記念パーティ LT  
2018/10/01 @hey株式会社

driller@patraqushe

### だれ？

* なまえ: どりらん
* 本名: driller

### しごと

* 本業: デリバティブのトレーダー
* 副業: 先生

## こんかいのおだい

Pythonの新しい書き方をまなぶ

### 人類にはまだ早い？
* 早めに使うことで当事者意識が高まる  
  生まれたときから知っている -> 記憶に残りやすい  
  あとで知る -> アレなんだったっけ？
* 技術の変化に耐性ができる  
  古い技術に固執しない
* ドヤれる

### 新しい書き方を知らない場合
`os.path`を使ってすべての拡張子を取得する処理

In [3]:
import os


def get_suffix(filepath):
    while True:
        filepath, suffix = os.path.splitext(filepath)
        if suffix:
            yield suffix
        else:
            break


suffixes = [x for x in get_suffix("dir/filename.tar.gz")][::-1]
print(suffixes)

['.tar', '.gz']


### 新しい書き方を知ってる場合
`pathlib`使おうぜ

In [4]:
import pathlib

pathlib.Path("dir/filename.tar.gz").suffixes

['.tar', '.gz']

## Data Classes

### Data Classes

* PEP557 で提案され、Python3.7で実装
* \_\_init\_\_() や \_\_repr\_\_() のような特殊メソッドを自動で追加
* メンバ変数は型アノテーションで定義できる

### namedtupleとの違い
* 属性をミュータブル(デフォルト)/イミュータブルにできる
* 通常のクラスとして扱える  
  namedtupleは継承が必用
* \_\_init\_\_の処理をカスタマイズできる

### 従来の書き方
#### Data Classesがないとき

In [5]:
class Data:
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

`__init__` の引数と `self.attr` の代入が冗長

#### Data Classesがあるとき

In [6]:
@dataclasses.dataclass
class Data37:
    name: str
    age: int

すっきり！

### \_\_repr\_\_の動作
#### Data Classesがないとき

In [7]:
Data("ノビタ", 11)

#### Data Classesがあるとき

In [8]:
Data37("シズカ", 11)

Data37(name='シズカ', age=11)

#### 従来のクラスで同じ機能を実装

In [9]:
class Data:
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

    def __repr__(self):
        return f"Data(name={self.name!r}, age={self.age!r})"

めんどくさい

### order
#### Data Classesがあるとき

In [10]:
@dataclasses.dataclass(order=True)
class Data37:
    name: str
    age: int


d1 = Data37("ノビタ", 11)
d2 = Data37("シズカ", 12)

print(d1 < d2)
sorted([d1, d2])

False


[Data37(name='シズカ', age=12), Data37(name='ノビタ', age=11)]

#### Data Classesがないとき

In [11]:
class Data:
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

    def __repr__(self):
        return f"Data(name={self.name!r}, age={self.age!r})"

    def __lt__(self, other):
        return (self.name, self.age) < (other.name, other.age)


d1 = Data37("ノビタ", 11)
d2 = Data37("シズカ", 12)

print(d1 < d2)
sorted([d1, d2])

False


[Data37(name='シズカ', age=12), Data37(name='ノビタ', age=11)]

### イミュータブル

In [12]:
@dataclasses.dataclass(frozen=True)
class Tax:
    rate: float = 0.08

tax = Tax()

In [16]:
tax.rate = 0.1

FrozenInstanceError: cannot assign to field 'rate'

### 辞書のキーに使える

In [13]:
{tax: "japan"}

{Tax(rate=0.08): 'japan'}

### mypyで型チェック

In [14]:
%%typecheck
# https://gist.github.com/knowsuchagency/f7b2203dd613756a45f816d6809f01a6
import dataclasses


@dataclasses.dataclass
class Data37:
    name: str
    age: int


data = Data37("ノビタ", "11")

<string>:12: error: Argument 2 to "Data37" has incompatible type "str"; expected "int"



### さいごに

本LTは __LT駆動学習__ です  

1. いずれは使ってみたい、学んでみたい技術がある
2. 発表できそうなイベントがある
3. 先に発表することを決めてから学習する

×: いつかやってみよう  
◯: やるキッカケを先に作ってしまう

### TBD
* init
  - \_\_post_init\_\_
  - dataclasses.InitVar
* ユーティリティ関数  
  - dataclasses.fields
  - dataclasses.asdict